In [1]:
!pip install transformers datasets evaluate rouge-score py7zr

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━

In [2]:
!pip install evaluate
import evaluate
import pandas as pd
import numpy as np
import shutil
from datasets import Dataset
from transformers import pipeline, BartTokenizer, BartForConditionalGeneration, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

In [3]:
import warnings
warnings.filterwarnings("ignore")


In [4]:
file_path =r"C:\Users\KIIT0001\Downloads\CompScholar_Dataset.csv"
df = pd.read_csv('CompScholar_Dataset.csv')


In [5]:
def describe_df(df):


    global categorical_features, continuous_features, binary_features

    categorical_features = [col for col in df.columns if df[col].dtype == 'object']
    binary_features = [col for col in df.columns if df[col].nunique() <= 2 and df[col].dtype != 'object']
    continuous_features = [col for col in df.columns if df[col].dtype != 'object' and col not in binary_features]

    print(f"\n{type(df).__name__} shape: {df.shape}")
    print(f"\n{df.shape[0]:,.0f} samples")
    print(f"\n{df.shape[1]:,.0f} attributes")
    print(f'\nMissing Data: \n{df.isnull().sum()}')
    print(f'\nDuplicates: {df.duplicated().sum()}')
    print(f'\nData Types: \n{df.dtypes}')

describe_df(df)


df = df.dropna(subset=['Document', 'Summary'])
continuous_features.remove('Paper Id')


import re

def clean_tags(text):
    clean = re.compile('<.*?>')
    clean = re.sub(clean, '', text)
    clean = '\n'.join([line for line in clean.split('\n') if not re.match('.*:\s*$', line)])
    return clean

def clean_df(df, cols):



    for col in cols:
        df[col] = df[col].fillna('').apply(clean_tags)
    return df


DataFrame shape: (371, 11)

371 samples

11 attributes

Missing Data: 
Paper Id         0
Paper Title      0
Key Words        0
Abstract         0
Conclusion       0
Document         0
Paper Type       0
Summary          0
Topic            0
OCR            126
labels           0
dtype: int64

Duplicates: 0

Data Types: 
Paper Id        int64
Paper Title    object
Key Words      object
Abstract       object
Conclusion     object
Document       object
Paper Type     object
Summary        object
Topic          object
OCR            object
labels         object
dtype: object


In [6]:
df = clean_df(df, ['Document', 'Summary'])

In [7]:
dataset = Dataset.from_pandas(df)

In [8]:
checkpoint = 'facebook/bart-large-xsum'
tokenizer = BartTokenizer.from_pretrained(checkpoint)
model = BartForConditionalGeneration.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/309 [00:00<?, ?B/s]

In [9]:
def preprocess_function(examples):
    inputs = [doc for doc in examples["Document"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["Summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/371 [00:00<?, ? examples/s]

In [10]:
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_val_split = train_test_split['train'].train_test_split(test_size=0.1)
train_ds = train_val_split['train']
val_ds = train_val_split['test']
test_ds = train_test_split['test']


In [11]:
!pip install evaluate


data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

metric = evaluate.load('rouge')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = ["\n".join(pred.strip().split()) for pred in decoded_preds]
    decoded_labels = ["\n".join(label.strip().split()) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [12]:
training_args = Seq2SeqTrainingArguments(
    output_dir='bart_summarization',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

#key = 77f8bf2273bb043420359f59eeaa9365b5244452

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 2329232 (2329232-kiit-deemed-to-be-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


AttributeError: 'numpy.float64' object has no attribute 'mid'

In [15]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = ["\n".join(pred.strip().split()) for pred in decoded_preds]
    decoded_labels = ["\n".join(label.strip().split()) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    '
    result = {key: value.mid.fmeasure * 100 if hasattr(value, 'mid') else value * 100 for key, value in result.items()}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [16]:
evaluation_results = trainer.evaluate(eval_dataset=test_ds)
print(evaluation_results)

Epoch,Training Loss,Validation Loss


AttributeError: 'numpy.float64' object has no attribute 'mid'

In [17]:
model_dir = "bart_finetuned_summarization"
trainer.save_model(model_dir)
tokenizer.save_pretrained(model_dir)

shutil.make_archive(model_dir, 'zip', model_dir)

'/content/bart_finetuned_summarization.zip'

In [ ]:
summarizer_pipeline = pipeline('summarization', model=model_dir)

example_texts = [
   "Multi-document Summarization via Deep Learning
Techniques: A SurveyMulti-document summarization (MDS), Deep learning models, Objective functions, Taxonomy, Evaluation metrics, Future directions, Information aggregationMulti-document summarization (MDS) is an effective tool for information aggregation that generates an informative and concise summary from a cluster of topic-related documents. Our survey, the first of its kind,
systematically overviews the recent deep-learning-based MDS models. We propose a novel taxonomy to summarize the design strategies of neural networks and conduct a comprehensive summary of the state of the
art. We highlight the differences between various objective functions that are rarely discussed in the existing
literature. Finally, we propose several future directions pertaining to this new and exciting field.In this article, we have presented the first comprehensive review of the most notable works to date
on deep-learning-based multi-document summarization (MDS). We propose a taxonomy for organizing and clustering existing publications and devise the network design strategies based on the
state-of-the-art methods. We also provide an overview of the existing multi-document objective
functions, evaluation metrics, and datasets and discuss some of the most pressing open problems
and promising future extensions in MDS research. We hope this survey provides readers with a
comprehensive understanding of the key aspects of MDS tasks, clarifies the most notable advances,
and sheds light on future studieslla i aye RR | poe [Sena Sena | Sena, —+ ar ea) } (b) Word/Sentence-level Concatenation MethodsInput Extractive Summarization Hybrid Summarization Selective Selective | {Generative| Extractor; —e| Selective || input —e) Extractor —e| Sects [Abstractor —+| Generative Extractive Abstractive Model Abstractive Summarization —= Sree Seat avast —+{ Generative Summary. ak Generative) (Generative) sector —{ Sena [avstraror} (Ser } Abstractive - Abstractive Model /‘Dec {a) Naive Networks (c} Auxiliary Task Networks Processing earner} (b) Ensemble Networks =a ao (6) Reconstruction Networks elton Groh === {e) Fusion Networks Encoder oecoser Processing (f) Graph Neural Networks Processing |! {suman (g) Encoder-Decoder Structure a ocesine a a (h) Pre-trained Language Models",
   "Another example document text..."
]

for text in example_texts:
   print(f"Original Text:\n{text}\n")
   print(f"Generated Summary:\n{summarizer_pipeline(text)}\n")